Import things


In [ ]:
import torch
import torchvision
import einops

from torch import nn
from IPython.display import display

pil = torchvision.transforms.ToPILImage()


: 

trying to use einops to split image into patches


In [9]:
mnist = torchvision.datasets.MNIST("dataset", download=True)
image = mnist.data[0]

display(pil(image))
display(image.shape)

result = einops.rearrange(
    image.reshape((28, 28)), "(pw w) (ph h) -> (pw ph) w h", pw=2, ph=2
)

for i in range(4):
    display(pil(result[i]))

# in the following code we use train_set or test_set instead of mnist.
del mnist


torch.Size([28, 28])

ok let's try other things with einops


In [10]:
from einops import rearrange

a = torch.rand((1, 28, 28))

rearrange(a, "b (pw w) (ph h) -> b (pw ph) (w h)", pw=4, ph=4).shape


torch.Size([1, 16, 49])

I got a problem: torch.cat doesn't work the same way as tensor addition. so we need to manually repeat the tensor in other channels.


In [11]:
a = torch.Tensor(((1, 1), (2, 2), (3, 3)))
a = a.reshape((1, 3, 2))
einops.repeat(a, "() h w -> repeat h w", repeat=3)


tensor([[[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.]]])

looks good to me

now, let's try to create a vision transformer


but before that, let's try to figure out how pytorch handles tensor addition


In [12]:
a = torch.Tensor(((1, 2), (3, 4), (5, 6)))
print(a)
b = torch.Tensor((1, 2))
print(b)

print(a + b)


tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
tensor([1., 2.])
tensor([[2., 4.],
        [4., 6.],
        [6., 8.]])


ok so it worked as i expected.


In [13]:
from einops.layers.torch import Rearrange


class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()

        # shape: (batch, 28, 28) == (batch, 4*7, 4*7) -> (batch, 4*4, 7*7)
        self.patch_embedding = nn.Sequential(
            Rearrange("b (pw w) (ph h) -> b (pw ph) (w h)", pw=4, ph=4),
            nn.Linear(7 * 7, 7 * 7),
        )
        # shape: (batch, 4*4+1, 7*7)
        self.class_token = nn.parameter.Parameter(torch.randn((1, 1, 7 * 7)))
        # shape: same
        self.position_encodings = nn.parameter.Parameter(
            torch.randn((1, 4 * 4 + 1, 7 * 7))
        )
        # the actual transformer, we use the transformer provided by pytorch here.
        # d_model is the dimention for the embedding vector
        # n_head is the number of heads in the multi head self attention thing
        # d_model has to be divisible by n_head
        # dim_feedforward is the number of hidden nodes in hidden layer in feedforward block
        # batch_first means the first dim of the tensor is batch
        # shape: same
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=7 * 7, nhead=7, dim_feedforward=128, batch_first=True
            ),
            3,
        )
        # the final MLP
        # shape: (batch, 4*4+1, 7*7) -> (batch, 10)
        self.mlp_head = nn.Sequential(nn.LayerNorm(7 * 7), nn.Linear(7 * 7, 10))

    def forward(self, x):
        x = self.patch_embedding(x)
        batch_size, _, _ = x.shape
        class_token = einops.repeat(
            self.class_token,
            "() words features -> repeat words features",
            repeat=batch_size,
        )
        x = torch.cat((class_token, x), dim=-2)
        x += self.position_encodings
        x = self.transformer_encoder(x)
        x = x[:, 0]
        x = self.mlp_head(x)
        return x


let's try to use this absolute abomination we've created


In [14]:
vision_transformer_model = VisionTransformer()
vision_transformer_model(torch.rand((1, 28, 28)))


tensor([[-0.0702, -0.4120, -0.4161,  0.0617,  0.1407, -0.6397,  0.7642, -0.6262,
         -0.5885, -0.1532]], grad_fn=<AddmmBackward0>)

ok, it works. And the shape seems to be alright.

now let's try to train it.


first, we need to load mnist data:


In [2]:
from torchvision import transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

train_set = torchvision.datasets.MNIST(
    "dataset", train=True, download=True, transform=transform
)
test_set = torchvision.datasets.MNIST("dataset", train=False, transform=transform)


100.0%


Extracting dataset\MNIST\raw\train-images-idx3-ubyte.gz to dataset\MNIST\raw



102.8%


Extracting dataset\MNIST\raw\train-labels-idx1-ubyte.gz to dataset\MNIST\raw



100.0%


Extracting dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to dataset\MNIST\raw



112.7%

Extracting dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset\MNIST\raw



then, let's write the code for training


In [15]:
def train(model, optimizer, scheduler, criterion, epochs, train_loader):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (x, y) in enumerate(train_loader):
            x = einops.rearrange(x, "b c h w -> (b c) h w")
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(
                    f"epoch: {epoch}, i: {batch_idx*len(x)}/{len(train_loader.dataset)}, loss: {loss.item()}"
                )

        scheduler.step()


training setup:


In [16]:
criterion = nn.CrossEntropyLoss()
epochs = 4
batch_size = 64
optimizer = torch.optim.Adam(vision_transformer_model.parameters(), lr=0.001)
# for every step, this scheduler times the lr by gamma
# step_size if how many steps before it times the lr by gamma
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)


and let's train the model:


In [17]:
train(
    vision_transformer_model,
    optimizer,
    scheduler,
    criterion,
    epochs,
    train_loader,
)


epoch: 0, i: 0/60000, loss: 2.347409963607788
epoch: 0, i: 640/60000, loss: 2.2308762073516846
epoch: 0, i: 1280/60000, loss: 2.1063649654388428
epoch: 0, i: 1920/60000, loss: 1.6644877195358276
epoch: 0, i: 2560/60000, loss: 1.372070074081421
epoch: 0, i: 3200/60000, loss: 1.1328141689300537
epoch: 0, i: 3840/60000, loss: 0.9578725695610046
epoch: 0, i: 4480/60000, loss: 0.8262578248977661
epoch: 0, i: 5120/60000, loss: 0.975385308265686
epoch: 0, i: 5760/60000, loss: 0.660971462726593
epoch: 0, i: 6400/60000, loss: 0.6545061469078064
epoch: 0, i: 7040/60000, loss: 0.5432608127593994
epoch: 0, i: 7680/60000, loss: 0.5247681140899658
epoch: 0, i: 8320/60000, loss: 0.4616728127002716
epoch: 0, i: 8960/60000, loss: 0.4974525570869446
epoch: 0, i: 9600/60000, loss: 0.5619866251945496
epoch: 0, i: 10240/60000, loss: 0.819754958152771
epoch: 0, i: 10880/60000, loss: 0.43658339977264404
epoch: 0, i: 11520/60000, loss: 0.6873295307159424
epoch: 0, i: 12160/60000, loss: 0.43437400460243225
epo

KeyboardInterrupt: 

looks like to be a great success to me

( actually, as i was creating this notebook, many things went wrong. But thanks to the magic of _editing_, everything seems to be fine :) )


Let's try to use the model to predict handwritings:
